# Attention based few-shot learning for movie recommendations

In [6]:
import pandas as pd
import pandas_profiling
import numpy as np
import os
from tqdm.notebook import tqdm
pd.options.mode.chained_assignment = None 
import torch
from torch import nn

### Load up MovieLens datasets

In [7]:
root_dir = '/Users/wend824/data/movies/movielens/ml-25m'
movies_fn = 'movies.csv'
rating_fn = 'ratings.csv'
movies_dir = os.path.join(root_dir, movies_fn)
rating_dir = os.path.join(root_dir, rating_fn)
movies_df = pd.read_csv(movies_dir, header=0)
rating_df = pd.read_csv(rating_dir, header=0)

In [3]:
user_test_df = rating_df.loc[rating_df['userId'] == ]

SyntaxError: invalid syntax (<ipython-input-3-d5c6043eef39>, line 1)

### Show me what you got...

In [8]:
movies_df.sample(5)

,movieId,title,genres
24817,122375,Gun for a Coward (1957),Western
22906,116493,The Returned (2013),Drama|Horror|Thriller
48467,176713,Frankenthumb (2002),Animation|Comedy|Horror|Sci-Fi
51807,183941,Return of the Blind Dead (1973),Horror
35593,148184,Father Jackleg (1974),Action|Comedy|Western


In [9]:
rating_df.sample(5)

,userId,movieId,rating,timestamp
24896801,161912,852,3.0,901453016
17380525,112588,70336,3.0,1263903591
19048282,123527,186447,3.0,1522795355
15742860,101941,253,3.0,844119758
6840072,44335,736,5.0,849998074


In [5]:
movies_profile = pandas_profiling.ProfileReport(movies_df)
movies_profile.to_file('./results/movies_profile_1.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
rating_profile = pandas_profiling.ProfileReport(rating_df[['userId', 'movieId', 'rating']])
rating_profile.to_file('./results/rating_profile_1.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Data Prep
  
### In There are 62,423 unique movieIds but only 62,325 unique titles. Additionaly, there are 59,047 unique movieIds in rating_df. Let's find the intersection and then deal with the remaining repeated titles.

In [10]:
movies_df2 = movies_df[movies_df.movieId.isin(rating_df.movieId)]
movies_df2 = movies_df2.reset_index(drop=True)

In [11]:
movies_profile2 = pandas_profiling.ProfileReport(movies_df2)
movies_profile2.to_file('./results/movies_profile_2.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Now we have 59047 unique movieIds and 58958 titles. Let's find which movieIds point to the same titles and convert them in both dataframes to have the same unique movieId

In [12]:
duplicates = movies_df2.title.duplicated(keep=False)
duplicate_movies = movies_df2[duplicates].sort_values("title")
duplicate_movies.head(n=10)

,movieId,title,genres
57751,204982,9 (2009),Comedy
13706,71057,9 (2009),Adventure|Animation|Sci-Fi
35223,151797,Absolution (2015),Drama|Thriller
29484,136696,Absolution (2015),Action|Adventure|Crime|Thriller
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
22108,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy
58355,206712,American Woman (2019),(no genres listed)
58554,207249,American Woman (2019),Drama
32242,143647,An Inspector Calls (2015),Drama|Thriller
32567,144450,An Inspector Calls (2015),Action|Comedy


In [14]:
print(len(duplicate_movies))

178


### It looks like we have 89 movies that were duplicated once each.

In [15]:
duplicates = movies_df2.title.duplicated(keep='first')
old_IDs = movies_df2[duplicates].sort_values("title")
replacements = movies_df2.title.duplicated(keep='last')
new_IDs = movies_df2[replacements].sort_values("title")

In [16]:
old_IDs2 = old_IDs.reset_index()
old_IDs2.head()

,index,movieId,title,genres
0,57751,204982,9 (2009),Comedy
1,35223,151797,Absolution (2015),Drama|Thriller
2,22108,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy
3,58554,207249,American Woman (2019),Drama
4,32567,144450,An Inspector Calls (2015),Action|Comedy


In [17]:
new_IDs2 = new_IDs.reset_index()
new_IDs2.head()

,index,movieId,title,genres
0,13706,71057,9 (2009),Adventure|Animation|Sci-Fi
1,29484,136696,Absolution (2015),Action|Adventure|Crime|Thriller
2,580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
3,58355,206712,American Woman (2019),(no genres listed)
4,32242,143647,An Inspector Calls (2015),Drama|Thriller


### Looks like there are 89 movies that are duplicated once but lets verify that the 'title' columns are identical.

In [18]:
new_IDs2['title'].equals(old_IDs2['title'])

True

### We can see inconsistent genres, so let's merge unique genres and exclude (no genres listed) if other genres are present. They look like strings...

In [19]:
print(type(new_IDs['genres'][13706]))
print(new_IDs['genres'][13706])

<class 'str'>
Adventure|Animation|Sci-Fi


### Let's start by finding any title's with matching genres then deal with the (no genres listed) entries then concatenate the rest.

In [20]:
sames = new_IDs2['genres']==old_IDs2['genres']
print(len(new_IDs2[sames]))
print(new_IDs2["genres"][sames])

12
10             Horror
11       Comedy|Drama
20             Comedy
27    Horror|Thriller
29           Thriller
30             Horror
33        Documentary
50           Thriller
56             Comedy
58              Drama
68             Comedy
80             Comedy
Name: genres, dtype: object


### This is good, since "(no genres listes)" is never present in both series, we know that every duplicate title has at least one genre. We can just concatenate all the strings, convert 'em to an OrederedDict, try removing the "(no genres listed)" key, then convert the keys back to strings.  Ezpz.

In [21]:
from collections import OrderedDict, defaultdict

concat_genres = new_IDs2["genres"].str.cat(old_IDs2['genres'], sep ="|")

# for row in concat_genres:
#     print(f"Row: {row}")

def clean_str(string):
    _D = OrderedDict.fromkeys(string)
    try:
        _D.pop("(no genres listed)")
    except:
        pass
    return _D.keys()
    
desired = concat_genres.str.split('|') \
                .apply(lambda x: clean_str(x)) \
                .str.join('|')

# for row in desired:
#     print(f"Row: {row}")

new_IDs2["genres"] = desired

### Finally, both the movies and ratings databases have the duplicate movieID's so we will change all of the ID's that point to the old movieId's (duplicates) to the new movieId's.
  
### Let's start with the movies dataframe. We will replace all of the new combined genres and remove all of the old movieId rows.

In [22]:
import copy

movies_df3 = copy.deepcopy(movies_df2)

for idx, row in new_IDs2.iterrows():
    movies_df3.at[row["index"], 'genres'] = row['genres']
    
print(len(movies_df3))

59047


### Now we drop the duplicates from the movies_df

In [23]:
drop_idx = old_IDs2['index']

movies_df4 = copy.deepcopy(movies_df3)
movies_df4 = movies_df4.drop(index=drop_idx)
movies_df4 = movies_df4.reset_index(drop=True)
print(len(movies_df4))
print(len(pd.unique(movies_df4['title'])))

58958
58958


In [24]:
# Just some testing
print(movies_df4.loc[movies_df4['movieId'] == 175849])
print(movies_df4.loc[movies_df4['title'] == 'WALL·E (2008)'])

       movieId                               title genres
45134   175849  Shirley: Visions of Reality (2013)  Drama
       movieId          title                                       genres
12423    60069  WALL·E (2008)  Adventure|Animation|Children|Romance|Sci-Fi


In [25]:
movies_df4.head(n=10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [26]:
movies_profile4 = pandas_profiling.ProfileReport(movies_df4)
movies_profile4.to_file('./results/movies_profile_4.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Lastly, we need to replace all of the movieId's in ratings that point to the old_IDs with the respective movieId from new_IDs:

In [27]:
Id_dict = {}
for idx, row in old_IDs2.iterrows():
    Id_dict[old_IDs2.iloc[idx]["movieId"]] = new_IDs2.iloc[idx]["movieId"]

In [28]:
# Get rid of redundancies in movieId
def replace_movieId(movie_Id, Id_dict):
    try:
        new_movie_Id = Id_dict[movie_Id]
        return new_movie_Id
    except:
        pass
    return movie_Id

In [29]:
rating_df2 = copy.deepcopy(rating_df[['userId', 'movieId', 'rating']])
rating_df2['movieId'] = rating_df2['movieId'].apply(lambda movie_Id: replace_movieId(movie_Id, Id_dict))

### We want non-zero integer ratings with low ratings (<=1/2*max) given negative values:

In [30]:
# Round ratings to discrete values
rating_df2['rating'] = ((rating_df2['rating'].values*2)-5).round()

In [31]:
rating_df2['rating'] = rating_df2['rating'].apply(lambda x: x-1 if x <= 0 else x)

In [32]:
np.sort(rating_df2['rating'].unique())

array([-5., -4., -3., -2., -1.,  1.,  2.,  3.,  4.,  5.])

### The movieId's do not span from 0-len(set), so lets remap the unique ID's to the integer indices. First, we will make a dict mapping the movieId's to the integer indices:

In [33]:
# Make movieId : index dict AND change movieId's in movies_df
mv_Id_dict = {}
for idx, row in movies_df4.iterrows():
    mv_Id_dict[ row["movieId"] ] = idx
    movies_df4.loc[idx, "movieId"] = idx

### Now we remap the movieId's in the rating_df:

In [34]:
rating_df3 = copy.deepcopy(rating_df2)
rating_df3['movieId'] = rating_df3['movieId'].apply(lambda x: mv_Id_dict[x])

### That should be the final touch on the movies_df

In [31]:
movies_profile4 = pandas_profiling.ProfileReport(movies_df4)
movies_profile4.to_file('./results/movies_profile_4.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### The unique movieId's in rating_df3 should match the movieId's in movies_df4:

In [35]:
rating_mv_Ids = rating_df3['movieId'].unique()
movie_df_mv_Ids = movies_df4['movieId'].unique()

In [36]:
print(type(rating_mv_Ids), len(rating_mv_Ids))
print(type(movie_df_mv_Ids), len(movie_df_mv_Ids))
rating_mv_Ids.sort()
movie_df_mv_Ids.sort()

<class 'numpy.ndarray'> 58958
<class 'numpy.ndarray'> 58958


In [38]:
np.array_equal(rating_mv_Ids, movie_df_mv_Ids)

True

### Cool, now we can create a dict mapping the movieId's to the titles:

In [39]:
movies_df4['movieId']
id2title_dict = dict(zip(movies_df4['movieId'], movies_df4['title']))

In [40]:
for mv_id, title in id2title_dict.items():
    print(mv_id, title)

0 Toy Story (1995)
1 Jumanji (1995)
2 Grumpier Old Men (1995)
3 Waiting to Exhale (1995)
4 Father of the Bride Part II (1995)
5 Heat (1995)
6 Sabrina (1995)
7 Tom and Huck (1995)
8 Sudden Death (1995)
9 GoldenEye (1995)
10 American President, The (1995)
11 Dracula: Dead and Loving It (1995)
12 Balto (1995)
13 Nixon (1995)
14 Cutthroat Island (1995)
15 Casino (1995)
16 Sense and Sensibility (1995)
17 Four Rooms (1995)
18 Ace Ventura: When Nature Calls (1995)
19 Money Train (1995)
20 Get Shorty (1995)
21 Copycat (1995)
22 Assassins (1995)
23 Powder (1995)
24 Leaving Las Vegas (1995)
25 Othello (1995)
26 Now and Then (1995)
27 Persuasion (1995)
28 City of Lost Children, The (Cité des enfants perdus, La) (1995)
29 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
30 Dangerous Minds (1995)
31 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
32 Wings of Courage (1995)
33 Babe (1995)
34 Carrington (1995)
35 Dead Man Walking (1995)
36 Across the Sea of Time (1995)
37 It Takes Two (1995)
38 Clueles

1037 Sleepers (1996)
1038 Sunchaser, The (1996)
1039 Johns (1996)
1040 Five Angles on Murder (1950)
1041 Shall We Dance (1937)
1042 Damsel in Distress, A (1937)
1043 Crossfire (1947)
1044 Murder, My Sweet (1944)
1045 Macao (1952)
1046 For the Moment (1994)
1047 Willy Wonka & the Chocolate Factory (1971)
1048 Sexual Life of the Belgians, The (Vie sexuelle des Belges 1950-1978, La) (1994)
1049 Innocents, The (1961)
1050 Sleeper (1973)
1051 Bananas (1971)
1052 Fish Called Wanda, A (1988)
1053 Monty Python's Life of Brian (1979)
1054 Victor/Victoria (1982)
1055 Candidate, The (1972)
1056 Great Race, The (1965)
1057 Bonnie and Clyde (1967)
1058 Old Man and the Sea, The (1958)
1059 Dial M for Murder (1954)
1060 Madame Butterfly (1995)
1061 Dirty Dancing (1987)
1062 Reservoir Dogs (1992)
1063 Platoon (1986)
1064 Weekend at Bernie's (1989)
1065 Basic Instinct (1992)
1066 Doors, The (1991)
1067 Crying Game, The (1992)
1068 Glengarry Glen Ross (1992)
1069 Sophie's Choice (1982)
1070 E.T. the Ext

1787 Deep Impact (1998)
1788 Little Men (1998)
1789 Woo (1998)
1790 Hanging Garden, The (1997)
1791 Lawn Dogs (1997)
1792 Quest for Camelot (1998)
1793 Godzilla (1998)
1794 Bulworth (1998)
1795 Fear and Loathing in Las Vegas (1998)
1796 Opposite of Sex, The (1998)
1797 I Got the Hook Up (1998)
1798 Almost Heroes (1998)
1799 Hope Floats (1998)
1800 Insomnia (1997)
1801 Little Boy Blue (1997)
1802 Ugly, The (1997)
1803 Perfect Murder, A (1998)
1804 Beyond Silence (Jenseits der Stille) (1996)
1805 Six Days Seven Nights (1998)
1806 Can't Hardly Wait (1998)
1807 Cousin Bette (1998)
1808 High Art (1998)
1809 Land Girls, The (1998)
1810 Passion in the Desert (1998)
1811 Children of Heaven, The (Bacheha-Ye Aseman) (1997)
1812 Dear Jesse (1997)
1813 Dream for an Insomniac (1996)
1814 Hav Plenty (1997)
1815 Henry Fool (1997)
1816 Marie from the Bay of Angels (Marie Baie Des Anges) (1997)
1817 Mr. Jealousy (1997)
1818 Mulan (1998)
1819 Resurrection Man (1998)
1820 X-Files: Fight the Future, The (

2537 Star Wars: Episode I - The Phantom Menace (1999)
2538 Love Letter, The (1999)
2539 Besieged (a.k.a. L' Assedio) (1998)
2540 Frogs for Snakes (1998)
2541 Saragossa Manuscript, The (Rekopis znaleziony w Saragossie) (1965)
2542 Mummy, The (1932)
2543 Mummy, The (1959)
2544 Mummy's Curse, The (1944)
2545 Mummy's Ghost, The (1944)
2546 Mummy's Hand, The (1940)
2547 Mummy's Tomb, The (1942)
2548 Mommie Dearest (1981)
2549 Superman (1978)
2550 Superman II (1980)
2551 Superman III (1983)
2552 Superman IV: The Quest for Peace (1987)
2553 Dracula (1931)
2554 House of Dracula (1945)
2555 House of Frankenstein (1944)
2556 Frankenstein (1931)
2557 Son of Frankenstein (1939)
2558 Ghost of Frankenstein, The (1942)
2559 Frankenstein Meets the Wolf Man (1943)
2560 Curse of Frankenstein, The (1957)
2561 Son of Dracula (1943)
2562 Wolf Man, The (1941)
2563 Howling II: Your Sister Is a Werewolf (1985)
2564 Tarantula (1955)
2565 Rocky Horror Picture Show, The (1975)
2566 Flying Saucer, The (1950)
2567

3537 Those Who Love Me Can Take the Train (Ceux qui m'aiment prendront le train) (1998)
3538 Vagabond (Sans toit ni loi) (1985)
3539 Moonraker (1979)
3540 Man with the Golden Gun, The (1974)
3541 King in New York, A (1957)
3542 Woman of Paris, A (1923)
3543 In Old California (1942)
3544 Fighting Seabees, The (1944)
3545 Dark Command (1940)
3546 Cleo from 5 to 7 (Cléo de 5 à 7) (1962)
3547 Big Momma's House (2000)
3548 Running Free (1999)
3549 Abominable Snowman, The (Abominable Snowman of the Himalayas, The) (1957)
3550 American Gigolo (1980)
3551 Anguish (Angustia) (1987)
3552 Blood Spattered Bride, The (La novia ensangrentada) (1972)
3553 City of the Living Dead (a.k.a. Gates of Hell, The) (Paura nella città dei morti viventi) (1980)
3554 Endless Summer, The (1966)
3555 Guns of Navarone, The (1961)
3556 Blow-Out (La grande bouffe) (1973)
3557 Lured (1947)
3558 Pandora and the Flying Dutchman (1951)
3559 Quatermass and the Pit (1967)
3560 Quatermass 2 (Enemy from Space) (1957)
3561 Pu

...

55022 Pagan Moon (1932)
55023 Bosko's Soda Fountain (1931)
55024 The Tree's Knees (1931)
55025 Yodeling Yokels (1931)
55026 End of Summer (2017)
55027 Knife in the Clear Water (2018)
55028 The Woman with Red Boots (1974)
55029 The Second Awakening of Christa Klages (1978)
55030 Conny Plank - The Potential of Noise (2017)
55031 Endless Waltz (1995)
55032 Make Me Up (2018)
55033 Un día vi 10000 elefantes (2015)
55034 Lots of Kids, a Monkey and a Castle (2017)
55035 The Widowed Witch (2017)
55036 Wretch (2019)
55037 The Little Witch (2018)
55038 Designs on Jerry (1955)
55039 Treasures from the Wreck of the Unbelievable (2017)
55040 Raid in St. Pauli (1932)
55041 Razzia (2018)
55042 The Homeboy (2018)
55043 Amnesia (2015)
55044 Tiempo de silencio (1986)
55045 Breakaway (1966)
55046 Frightmare (1983)
55047 Backyard (1984)
55048 The Least of These (2019)
55049 Rigged (2008)
55050 The Golem (2018)
55051 Untogether (2019)
55052 Berlin, I Love You (2019)
55053 The Isle (2018)
55054 Love, Cecil 

55916 The Legend of Time (2006)
55917 B (2015)
55918 Tres dies amb la família (2009)
55919 La barraca (1945)
55920 Mr. Gaga (2015)
55921 Torero (1957)
55922 Bobby Robson: More Than a Manager (2018)
55923 Raza, el espíritu de Franco (1977)
55924 The Prodigious Life of Father Vincent (1978)
55925 Atilano, presidente (1998)
55926 Diamond Flash (2012)
55927 The Legend of Cocaine Island (2019)
55928 Lucky (2006)
55929 ReMastered: The Miami Showband Massacre (2019)
55930 Omerta (2008)
55931 The Doll (2017)
55932 Matriarch (2018)
55933 Where Man Returns (2019)
55934 Weaving the Past: Journey of Discovery (2014)
55935 Full Contact (1992)
55936 An Honorable Young Man (1963)
55937 The Pact (2018)
55938 Microhabitat (2018)
55939 Snapshots (2018)
55940 Tempting Devils (2018)
55941 The Burial of Kojo (2018)
55942 The Russian Bride (2019)
55943 Diane (2019)
55944 Vampires: Los Muertos (2002)
55945 Painted Faces (1988)
55946 Dark, Almost Night (2019)
55947 Aladdin (2019)
55948 Necromancer (1988)
5594

56728 Pope Vs. Hitler (2017)
56729 School's Out (2019)
56730 A Date in 2025 (2017)
56731 The Arrival (2017)
56732 Whenever You're Ready (2018)
56733 Cul-De-Sac (2016)
56734 I Know You from Somewhere (2017)
56735 Assimilate (2019)
56736 Cabin Fever (2000)
56737 Chetzemoka's Curse (2001)
56738 Walking with Cavemen (2003)
56739 Wings of Life (2011)
56740 Vándorszínészek (2018)
56741 Once Upon a Time in China IV (1993)
56742 Avengement (2019)
56743 Les Misérables (2019)
56744 Atlantics (2019)
56745 Elvira's Haunted Hills (2002)
56746 The Legend of 5 Mile Cave (2019)
56747 Photograph (2019)
56748 Winnie the Pooh: A Very Merry Pooh Year (2002)
56749 The Price of Honor (2014)
56750 Kung Fu Panda: Secrets of the Scroll (2016)
56751 Child's Play (2019)
56752 The Last Best Sunday (1999)
56753 The Last Resort (2018)
56754 Game of Thrones: The Last Watch (2019)
56755 India's Most Wanted (2019)
56756 Othon (1971)
56757 Cinderella and the Secret Prince (2018)
56758 Plan Bee (2007)
56759 Little Bee (

57441 One Piece: Baron Omatsuri and the Secret Island (2005)
57442 Night Must Fall (1964)
57443 The Olsen Gang in a Fix (1969)
57444 The Olsen Gang's Big Score (1972)
57445 The Olsen Gang Goes Crazy (1973)
57446 The Olsen Gang Outta Sight (1977)
57447 The Olsen Gang Goes to War (1978)
57448 The Last Exploits of the Olsen Gang (1974)
57449 Honeyland (2019)
57450 Captain Black (2019)
57451 Our New President (2018)
57452 Blonde Animals (2019)
57453 Yara (2018)
57454 The Disappearance of My Mother (2019)
57455 Murder Me, Monster (2019)
57456 Anjaneya (2003)
57457 Game Changers: Inside the Video Game Wars (2019)
57458 Voyage to Metropolis (2010)
57459 Sources of Life (2013)
57460 Chuchotage (2018)
57461 Gone with the Woman (2007)
57462 A Place On Earth (2013)
57463 Puss 'N Boots Travels Around the World (1976)
57464 Kharms (2017)
57465 Extraction Day (2014)
57466 Fail State (2018)
57467 Strawinsky and the Mysterious House (2012)
57468 Netflix Live (2017)
57469 Кикос (1979)
57470 Wavelength 

58551 The Winter Stallion (1992)
58552 Lovely Trash (2019)
58553 Unearthed & Untold: The Path to Pet Sematary (2017)
58554 Fingerpori (2019)
58555 Now Something Is Slowly Changing (2019)
58556 Hard Paint (2019)
58557 May (2008)
58558 The Immortal Woman (1993)
58559 Boys Will Be Boys (1976)
58560 Halt, Or I'll Miss! (1998)
58561 Odcházení (2011)
58562 Kristián (1939)
58563 Sun, Hay, Erotics (1991)
58564 Krakonoš a lyžníci (1981)
58565 Imperial and Royal Field Marshal (1930)
58566 The Good Soldier Svejk (1955)
58567 Snowboarďáci (2004)
58568 The Magic Quill (2018)
58569 Homeward (2019)
58570 The Border Fence (2018)
58571 In This Gray Place (2019)
58572 Knife in the Head (1978)
58573 Cinderela Baiana (1998)
58574 Eleições (2018)
58575 Trespassers (2018)
58576 The Fall of Fujimori (2006)
58577 Batla House (2019)
58578 Tell Me Who I Am (2019)
58579 Os Saltimbancos Trapalhões (1981)
58580 Os Trapalhões no Reino da Fantasia (1985)
58581 Os Fantasmas Trapalhões (1987)
58582 O Casamento dos Tra

### That should do it for our movies_df and rating_df. It is worth noting that movieId's start from 0 while userId's start from 1. That shouldn't matter.

In [36]:
rating_df3_profile = pandas_profiling.ProfileReport(rating_df3)
rating_df3_profile.to_file('./results/rating_profile_2.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
most_rated = [356,318,296,593,2571,260,480,527,110]
for i in most_rated:
    print(movies_df4.loc[[i]])

     movieId                          title                genres
356      356  It Could Happen to You (1994)  Comedy|Drama|Romance
     movieId                        title        genres
318      318  Swimming with Sharks (1995)  Comedy|Drama
     movieId             title genres
296      296  Quiz Show (1994)  Drama
     movieId                                      title genres
593      593  Wooden Man's Bride, The (Yan shen) (1994)  Drama
      movieId                                title  genres
2571     2571  It Came from Beneath the Sea (1955)  Sci-Fi
     movieId                     title genres
260      260  Ladybird Ladybird (1994)  Drama
     movieId                    title                           genres
480      480  Last Action Hero (1993)  Action|Adventure|Comedy|Fantasy
     movieId              title               genres
527      527  Serial Mom (1994)  Comedy|Crime|Horror
     movieId                                       title  \
110      110  Rumble in the Bronx (H

## IoU Similarity:

#### The paper uses an intersection over union of users that positively interact with two movies as a baseline similarity metric. The metric uses Baye's rule and an assumption of independence to determine that the probability of a user liking an item given they liked another item is given by : (equation 4).  
  
#### They also state that |U| is a constant and therfore not relevant. However, this will result in poorly normalized inputs to the deep model and will cause high variance. Additionally, more popular movies (i.e. the number of people that have rated the movie) will have a positive bias. This is easily dealt with by recognizing that users that have not rated either movie are not relevant and the equation should instead be : |Ui+nUj+|/|Ui+|+|Uj+|-|Ui+nUj+|

### Let's make sets of users that positively interact with a movie and sets of users that negatively interact with a movie:

In [40]:
pos_interactions = defaultdict(set)
neg_interactions = defaultdict(set)

pos_int_df = rating_df3[rating_df3['rating']>0]
neg_int_df = rating_df3[rating_df3['rating']<0]
print(len(pos_int_df))
print(len(neg_int_df))
pos_int_df = pos_int_df.drop(['rating'], axis=1)
neg_int_df = neg_int_df.drop(['rating'], axis=1)

20527057
4473038


In [41]:
pos_int_df.head()

,userId,movieId
0,1,292
1,1,302
2,1,303
3,1,654
4,1,878


In [42]:
pos_movie_user_df = pos_int_df.groupby(['movieId'])['userId'].agg(size=len, userSet=lambda x: set(x))
neg_movie_user_df = neg_int_df.groupby(['movieId'])['userId'].agg(size=len, userSet=lambda x: set(x))

In [43]:
pos_movie_user_df.head()

,size,userSet
movieId,,
0,52650,"{2, 3, 4, 5, 131074, 131077, 8, 10, 12, 13, 13..."
1,18826,"{131076, 98309, 131077, 9, 65545, 98313, 98316..."
2,9032,"{131075, 65540, 65541, 8, 32778, 131087, 23, 9..."
3,1656,"{49152, 24577, 49160, 122890, 139282, 139287, ..."
4,8777,"{65541, 65545, 32778, 131084, 131087, 18, 6555..."


In [44]:
neg_movie_user_df.head()

,size,userSet
movieId,,
0,4659,"{106497, 57350, 147463, 40968, 81931, 40983, 8..."
1,5402,"{16385, 114689, 155652, 24581, 32774, 73735, 9..."
2,2772,"{49152, 8195, 24579, 98312, 12, 57356, 24592, ..."
3,867,"{43008, 59393, 69632, 4100, 24581, 65540, 9626..."
4,2937,"{32769, 8195, 49165, 98329, 73754, 32798, 9014..."


In [45]:
"""
movie_user_dict: { movieId : { 'size' : int(len(userSet)), 'userSet' : set(userId's) } }
"""

pos_mv_dict = pos_movie_user_df.to_dict('index')
neg_mv_dict = neg_movie_user_df.to_dict('index')

### Make and IoU function using the sets:

In [46]:
# Use IoU of users that positively rate two different movies as a baseline of similarity
def getIoU(query, support, movie_user_dict):
    query_set= movie_user_dict[query]['userSet']
    support_set= movie_user_dict[support]['userSet']
    intersect_len = len(query_set.intersection(support_set))
    query_len = len(movie_user_dict[query]['userSet'])
    support_len = len(movie_user_dict[support]['userSet'])
    ratio = intersect_len/(query_len+support_len-intersect_len)
    return ratio

### Now let's test it out:

In [50]:
toy_story = 0
wall_e = 12423
heat = 5

ratio_good = getIoU(toy_story,wall_e,pos_mv_dict)
ratio_bad = getIoU(toy_story,heat,pos_mv_dict)

print(f'hits between Toy Story and Wall-e: {ratio_good}')
print(f'hits between Toy Story and Heat:    {ratio_bad}')

hits between Toy Story and Wall-e: 0.2001417696551299
hits between Toy Story and Heat:    0.19695410578308295


### I expected that to work better. This makes sense as all three are critically aclaimed and will be on many peoples' "good" list, lets try again with some more distinct movie demographics in mind:

In [51]:
too_fast_2_furiou = 6264
gone_in_60_s = 3616
the_notebook = 7851

ratio_good = getIoU(too_fast_2_furiou, gone_in_60_s, pos_mv_dict)
ratio_bad = getIoU(too_fast_2_furiou, the_notebook, pos_mv_dict)

print(f'hits between 2F2F and G60S:    {ratio_good}')
print(f'hits between 2F2F and The NB:  {ratio_bad}')

hits between 2F2F and G60S:    0.1101676698194325
hits between 2F2F and The NB:  0.058093994778067884


# IoU round 2
  
### As it turns out, using sets to calculate the IoU does not scale well and created a bottleneck in our network. Here I am going to vectorize the IoU calculation to improve speed.  
  
### Let's transfer the dicts above to numpy arrays:

In [56]:
# array shape: (# movies, # users)
pos_interaction_array = np.zeros((len(movies_df4),len(rating_df3['userId'].unique())))

for movieId, value in tqdm(pos_mv_dict.items()):
    userId_set = value['userSet']
    for user in userId_set:
        _user = user-1
        pos_interaction_array[movieId][_user] = 1

neg_interaction_array = np.zeros((len(movies_df4),len(rating_df3['userId'].unique())))

for movieId, value in tqdm(neg_mv_dict.items()):
    userId_set = value['userSet']
    for user in userId_set:
        _user = user-1
        neg_interaction_array[movieId][_user] = 1

  0%|          | 0/53039 [00:00<?, ?it/s]

  0%|          | 0/45155 [00:00<?, ?it/s]

In [58]:
def np_IoU(query_array, support_array):
    
    intersect = np.dot(query_array, support_array)
    query_len = np.sum(query_array)
    support_len = np.sum(support_array)
    
    IoU = intersect/(query_len+support_len-intersect)
    
    return IoU

In [61]:
ratio_good = np_IoU(pos_interaction_array[toy_story],pos_interaction_array[wall_e])
ratio_bad = np_IoU(pos_interaction_array[toy_story],pos_interaction_array[heat])

print(f'hits between Toy Story and Wall-e: {ratio_good}')
print(f'hits between Toy Story and Heat:    {ratio_bad}')

hits between Toy Story and Wall-e: 0.2001417696551299
hits between Toy Story and Heat:    0.19695410578308295


In [62]:
ratio_good = np_IoU(pos_interaction_array[too_fast_2_furiou], pos_interaction_array[gone_in_60_s])
ratio_bad = np_IoU(pos_interaction_array[too_fast_2_furiou], pos_interaction_array[the_notebook])

print(f'hits between 2F2F and G60S:    {ratio_good}')
print(f'hits between 2F2F and The NB:  {ratio_bad}')

hits between 2F2F and G60S:    0.1101676698194325
hits between 2F2F and The NB:  0.058093994778067884


### Looks good. Lets verify that this is faster too:

In [63]:
%timeit ratio_bad = getIoU(too_fast_2_furiou, the_notebook, pos_mv_dict)
%timeit ratio_bad = np_IoU(pos_interaction_array[too_fast_2_furiou], pos_interaction_array[the_notebook])

63.5 µs ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
108 µs ± 6.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Lol.  
  
### Let's see if less sparsely rated movies are faster with vectorization:

In [64]:
%timeit ratio_good = getIoU(too_fast_2_furiou, gone_in_60_s, pos_mv_dict)
%timeit ratio_good = np_IoU(pos_interaction_array[toy_story],pos_interaction_array[wall_e])

81 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
106 µs ± 6.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### I guess I will just leave the set based IoU in place

### Being that we will be utilizing few-shot learning, we need a positive and negative set for each user. The original paper uses a "top-k" dataset and considers the entire set as positives while using items excluded by the set as negative examples.  
  
### The dataset I am using has ratings (from 1-10), so I will consider ratings from (6-10) as positive examples and ratings from (1-5) as negative examples. For the first pass, I will implement the exact paper conditions and will only use samples with a rating between 6-10 as our positive set.

In [69]:
# This one takes a while, should be 15-20 seconds
user_df = rating_df3.groupby('userId')[['movieId','rating']].apply( \
          lambda g: list(map(list, g.values.tolist())))

In [70]:
# Split the movies into positive and negative sets per user
def split_labels(_list):
    '''
    Args:
        _list: list of tuple(movieID, rating)
    Returns:
        rate_dict: nested dictionary with structure: 
            dict(key = {'pos','neg'} : value = {dict(key = 'movieID' : value = rating)})
    '''
    pos_dict = defaultdict(int)
    neg_dict = defaultdict(int)
    rate_dict = defaultdict(dict)
    for tup in _list:
        if tup[1] < 0:
            neg_dict[int(tup[0])] = int(tup[1])
        else:
            pos_dict[int(tup[0])] = int(tup[1])
    rate_dict['pos'] = pos_dict
    rate_dict['neg'] = neg_dict
    return rate_dict

user_posneg = user_df.apply(lambda _list: split_labels(_list)).to_dict()

### Let's get the movies_df to a dict for saving

In [90]:
mv_df = movies_df4.set_index('movieId')
movies_dict = mv_df.to_dict('index')
movieId_set = set(movies_dict.keys())

NameError: name 'movies_df4' is not defined

In [74]:
md_keys = list(movies_dict.keys())
print(f'number of movies: {len(md_keys)}')
print(md_keys[0], movies_dict[md_keys[0]])

number of movies: 58958
0 {'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}


### Split into test/train splits

In [76]:
from sklearn.model_selection import train_test_split

s = pd.Series(user_posneg)
train_data , test_data  = [i.to_dict() for i in train_test_split(s, train_size=0.8)]

In [77]:
train_keys = list(train_data.keys())
print(len(train_keys))

print(train_keys[1], train_data[train_keys[1]])

130032
37922 defaultdict(<class 'dict'>, {'pos': defaultdict(<class 'int'>, {9: 1, 49: 5, 257: 3, 312: 1, 314: 5, 324: 1, 337: 3, 340: 5, 351: 3, 372: 3, 375: 3, 400: 3, 470: 3, 475: 3, 522: 3, 536: 1, 578: 3, 579: 1, 580: 1, 581: 5, 584: 1, 637: 3, 662: 3, 764: 3, 988: 1, 1013: 5, 1062: 1, 1099: 3, 1152: 1, 1159: 3, 1166: 3, 1167: 1, 1168: 3, 1170: 3, 1179: 3, 1183: 3, 1207: 5, 1226: 5, 1232: 3, 1237: 3, 1241: 5, 1242: 5, 1258: 3, 1321: 1, 1338: 1, 1360: 3, 1489: 1, 1523: 3, 1552: 5, 1559: 1, 1579: 5, 1628: 3, 1666: 5, 1701: 3, 1834: 5, 1868: 3, 1872: 3, 1913: 1, 1922: 3, 1923: 3, 1939: 3, 2015: 1, 2034: 1, 2177: 3, 2230: 1, 2262: 3, 2305: 5, 2364: 1, 2480: 5, 2481: 1, 2490: 1, 2537: 3, 2614: 1, 2670: 5, 2749: 5, 2766: 1, 2824: 3, 2826: 3, 2905: 3, 2967: 3, 3011: 1, 3015: 3, 3082: 3, 3160: 5, 3170: 3, 3208: 3, 3224: 5, 3291: 1, 3313: 3, 3314: 3, 3374: 3, 3429: 5, 3479: 5, 3524: 5, 3596: 5, 3598: 1, 3692: 3, 3697: 1, 3776: 1, 3779: 3, 3781: 1, 3844: 1, 3873: 3, 3890: 5, 3892: 5, 3914: 

In [78]:
train_count = 0
for k,v in train_data.items():
    train_count += len(v['pos'])
print(type(train_data), len(train_data), print(train_count))
print()
test_count = 0
for k,v in test_data.items():
    test_count += len(v['pos'])
print(type(test_data), len(test_data), print(test_count))
print()
tot = train_count + test_count
print(f'train ratio: {train_count/tot:.3f}')
print(f'test ratio: {test_count/tot:.3f}')

16392567
<class 'dict'> 130032 None

4134072
<class 'dict'> 32509 None

train ratio: 0.799
test ratio: 0.201


In [79]:
import pickle

# set of all movieId's
pickle.dump( movieId_set, open( "./data/movieId_set.p", "wb" ) )
# nested dict: [user]['pos'/'neg'][movieId] -> rating
pickle.dump( train_data, open( "./data/user_rating_dict_train.p", "wb" ) )
pickle.dump( test_data, open( "./data/user_rating_dict_test.p", "wb" ) )
# Dict movieId : 'title', 'genres'
pickle.dump( movies_dict, open( "./data/movies_dict.p", "wb") )
# Dict with movieId : 'size', 'userSet'
pickle.dump( pos_mv_dict, open( "./data/pos_movie2user_dict.p", "wb") )
pickle.dump( neg_mv_dict, open( "./data/neg_movie2user_dict.p", "wb") )

### unlike Harry, I'm not a wizard. Time to do some proto-prototyping:

In [80]:
import torch
import copy
'''So if A and B are of shape (3, 4), torch.cat([A, B], dim=0) will be of shape (6, 4) 
                              and torch.stack([A, B], dim=0) will be of shape (2, 3, 4).
'''
A = torch.tensor([[1,0,0]])
B = torch.tensor([[0,1,0]])
C = torch.tensor([[0,0,1]])

print(f'A: {A}, shape: {A.shape}')
print(f'B: {B}, shape: {B.shape}')
print(f'C: {C}, shape: {C.shape}')

catABC = torch.cat([A,B,C], dim=0)
catABC_T = torch.transpose(catABC, 0, 1).clone()

print()
print("original")
print(catABC)
print(catABC.shape)
print()
print("Transposed")
print(catABC_T)
print(catABC_T.shape)
print()
out = torch.mm(catABC, catABC_T)
print("MM output")
print(out)
print(out.shape)
catABC[0,0] = 5
print(catABC)
print(catABC_T)

A: tensor([[1, 0, 0]]), shape: torch.Size([1, 3])
B: tensor([[0, 1, 0]]), shape: torch.Size([1, 3])
C: tensor([[0, 0, 1]]), shape: torch.Size([1, 3])

original
tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])
torch.Size([3, 3])

Transposed
tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])
torch.Size([3, 3])

MM output
tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])
torch.Size([3, 3])
tensor([[5, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])
tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])


In [81]:
m = torch.nn.Softmax(dim=0)
input = torch.randn(3, 3)
input = torch.sum(input, dim=1)
print(input)
output = m(input)
print(output)

tensor([-1.2844, -2.6740, -0.4249])
tensor([0.2769, 0.0690, 0.6541])


# THE MODEL  
### Let's build the individual modules and test them first

In [193]:
class Embedder(nn.Module):
    def __init__(self, 
                 n_factors=4,
                 n_movies =58958):
        """
        Parameters
        ----------
        n_factors : int
            Size of embedding (latent factors).
        n_movies : int
            Number of movies to consider for user.

        Forward:
            x : tensor(k)
                tensor of movieId's for few-shot support
            
        Output
        ----------
        embeddings : tensor(k,n_factors)
            embedding tensors for movieId's used for support
        """
        super().__init__()
        self.embed = nn.Embedding(n_movies, n_factors)
    def forward(self, x):
        return self.embed(x)

class AttentionModule(nn.Module):
    def __init__(self, embedder):
        """
        Parameters
        ----------
        n_factors : int
            Size of embedding (latent factors).
        embedder  : class Embedder 
            pytorch embedding module
            
        Forward:
            x : tensor(k)
                tensor of movieId's for few-shot support
            
        Output
        ----------
        Support set weights : torch.tensor
            stochastic weighting of support set for linear combination of similarity values
        """
        super(AttentionModule, self).__init__()
        self.emb = embedder
        self.sm = torch.nn.Softmax(dim=0)

    def forward(self, x):
        x_T = x.clone()
        embed = self.emb(x)
        embed_T = self.emb(x_T)
        embed_T = torch.transpose(embed_T, 0, 1)
        dot = torch.mm(embed, embed_T)
        dot_sum = torch.sum(dot, dim=1)
        out = self.sm(dot_sum)
        
        return out

class SimilarityFeeder(nn.Module):
    def __init__(self, embedder, movie_user_dict):
        """
        Parameters
        ----------
        embedder : class Embedder
            generates movie embeddings for similarity network
        movie_user_dict: dict('movieId' : dict ( 'size' : int(set_size), 'userSet' : set (UserId's)))
            nested dict with movieId key and set of users that positively interacted with the movie
            
        Forward:
            query : Tensor(1)
                movieId for query movie
            support_set: Tensor(k)
                tensor of positive movieId's used for few-shot support
            
        Output
        ----------
        
        """
        super(SimilarityFeeder, self).__init__()
        
        self.movie_user_dict = movie_user_dict
        self.embedder = embedder
        
    def getIoU(self, query, support):
        """Gets Intersection over Union of positively interacting users between query and support"""
        
        # In cases where no users have rated a movie positively, IoU = 0
        try:
            query_set= self.movie_user_dict[query.item()]['userSet']
            support_set= self.movie_user_dict[support.item()]['userSet']

            intersect_len = len(query_set.intersection(support_set))
            query_len = self.movie_user_dict[query.item()]['size']
            support_len = self.movie_user_dict[support.item()]['size']

            ratio = intersect_len/(query_len+support_len-intersect_len)
        except:
            ratio = 0
        
        return ratio
    
    def makeIoUtensor(self, query, support_set):
        """Builds the tensor of IoU values"""
        
        iou_tens = torch.zeros(len(support_set))
        
        for idx, mv_id in enumerate(support_set):
            _IoU = self.getIoU(query, mv_id)
            iou_tens[idx] = _IoU
        
        return iou_tens
    
    def forward(self, query, support_set):
        
        support_embeds = self.embedder(support_set)
        #print("support embeds:", support_embeds.shape)
        query_embed = self.embedder(query)
        
        query_embeds = query_embed.repeat(1,len(support_set)).view(-1, support_embeds.shape[1])
        #print("query embeds:", query_embeds.shape)
        
        cat_embeds = torch.cat([support_embeds, query_embeds], dim=1)
        #print("cat embeds:", cat_embeds.shape)
        
        iou_tens = self.makeIoUtensor(query, support_set)
        
        return cat_embeds, iou_tens.unsqueeze(1)
        

# We will do the embedding on the outside so we can pass the whole support set as a batch
class SimilarityModule(nn.Module):
    def __init__(self, n_factors=32):
        """
        Parameters
        ----------
        n_factors : int
            size of movie embeddings
        x   : Tensor(n_factors*2)
            concatenatenated query embedding + support embedding
        IoU : Tensor(1)
            IoU value for users that interact with query and support
        Output
        ----------
        sig_out: Tensor(1)
            similarity value between movies
        """
        super(SimilarityModule, self).__init__()
        
        self.input_size = 2*n_factors
        self.hidden_size = n_factors
        
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, int(self.hidden_size/2))
        self.fc3 = torch.nn.Linear(int(self.hidden_size/2), 1)
        self.sigmoid = torch.nn.Sigmoid()
        
        
    def forward(self, x, IoU):
        hidden_1 = self.fc1(x)
        relu_1 = self.relu(hidden_1)
        hidden_2 = self.fc2(relu_1)
        relu_2 = self.relu(hidden_2)
        mlp_out = self.fc3(relu_2)
        sig = self.sigmoid(mlp_out)
        
        output = sig + IoU
        
        return output

# Test Modules

In [189]:
# inputs = torch.tensor([[60,1,42,23,84,45,26,74],
#                        [60,1,42,23,84,45,26,74]])
# IoUs = torch.tensor([[0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5],
#                      [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]])

inputs = torch.tensor([60,1,42,23,84,45,26,74])
IoUs = torch.tensor([0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5])

attention_factor = 4
attention_embedder = Embedder(n_factors=attention_factor, n_movies=100)
attention_embeds = attention_embedder(inputs)

similarity_factor = 8
similarity_embedder = Embedder(n_factors=similarity_factor, n_movies=100)
similarity_embeds = similarity_embedder(inputs)

query_embed = similarity_embedder(torch.tensor([[2],[1]]))

In [190]:
print(attention_embeds)
print(attention_embeds.shape)
print(attention_embeds.unsqueeze(1).shape)
print(attention_embeds.unsqueeze(1))
print()
print()
print(query_embed)
query_rep = query_embed.repeat(1,len(inputs)).view(-1, 8)
print(query_rep)
print(query_rep.shape)
print(similarity_embeds.shape)
cat_embs = torch.cat([similarity_embeds, query_rep], dim=1)
print()
print(cat_embs.shape)
print(cat_embs)
print()
# print(similarity_embeds)
# print(similarity_embeds.shape[1])
# print(similarity_embeds.unsqueeze(1).shape)
# print(similarity_embeds.unsqueeze(1))

tensor([[ 1.0751,  0.1073, -0.9735, -1.0010],
        [ 1.3783, -0.0343, -0.0926, -0.3799],
        [ 0.7013, -0.5418, -0.8006, -0.8500],
        [-0.0100, -0.2267, -0.1650,  1.1646],
        [ 1.7601, -0.6997,  1.0169,  0.5508],
        [-0.8775, -1.6113,  0.4134, -0.9172],
        [ 0.1831, -0.8970,  1.7812,  1.5801],
        [-1.7177, -0.8991,  1.6873,  0.8500]], grad_fn=<EmbeddingBackward>)
torch.Size([8, 4])
torch.Size([8, 1, 4])
tensor([[[ 1.0751,  0.1073, -0.9735, -1.0010]],

        [[ 1.3783, -0.0343, -0.0926, -0.3799]],

        [[ 0.7013, -0.5418, -0.8006, -0.8500]],

        [[-0.0100, -0.2267, -0.1650,  1.1646]],

        [[ 1.7601, -0.6997,  1.0169,  0.5508]],

        [[-0.8775, -1.6113,  0.4134, -0.9172]],

        [[ 0.1831, -0.8970,  1.7812,  1.5801]],

        [[-1.7177, -0.8991,  1.6873,  0.8500]]], grad_fn=<UnsqueezeBackward0>)


tensor([[[-1.4593,  0.3502,  0.4130, -0.4954,  0.7655, -0.3619,  1.1422,
          -0.2505]],

        [[ 0.2661, -0.6650,  0.0897,  2.28

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor

### Test AttentionModule

In [16]:
attention_module = AttentionModule(attention_embedder)

In [17]:
%timeit out = attention_module(inputs)

66.8 µs ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
out = attention_module(inputs)

In [19]:
print(out)

tensor([5.8494e-01, 1.3447e-03, 1.6588e-03, 3.5092e-01, 2.7581e-05, 8.3323e-07,
        1.7379e-05, 6.1093e-02], grad_fn=<SoftmaxBackward>)


### Test SimilarityModule

In [66]:
similarity_module = SimilarityModule(n_factors=similarity_factor)

In [71]:
IoUs = torch.tensor([-4,-3,-2,-1,1,2,3,4]).unsqueeze(1)
print(IoUs.shape)
print(cat_embs.shape)

torch.Size([2, 8])
torch.Size([8, 16])


In [72]:
%timeit sim_out = similarity_module(cat_embs,IoUs)

RuntimeError: The size of tensor a (8) must match the size of tensor b (2) at non-singleton dimension 0

In [69]:
sim_out = similarity_module(cat_embs, IoUs)

In [70]:
print(sim_out)

tensor([[[-3.4817, -2.4817, -1.4817, -0.4817,  1.5183,  2.5183,  3.5183,
           4.5183],
         [-3.4867, -2.4867, -1.4867, -0.4867,  1.5133,  2.5133,  3.5133,
           4.5133],
         [-3.4826, -2.4826, -1.4826, -0.4826,  1.5174,  2.5174,  3.5174,
           4.5174],
         [-3.4859, -2.4859, -1.4859, -0.4859,  1.5141,  2.5141,  3.5141,
           4.5141],
         [-3.4804, -2.4804, -1.4804, -0.4804,  1.5196,  2.5196,  3.5196,
           4.5196],
         [-3.4862, -2.4862, -1.4862, -0.4862,  1.5138,  2.5138,  3.5138,
           4.5138],
         [-3.4872, -2.4872, -1.4872, -0.4872,  1.5128,  2.5128,  3.5128,
           4.5128],
         [-3.4862, -2.4862, -1.4862, -0.4862,  1.5138,  2.5138,  3.5138,
           4.5138]],

        [[-3.4817, -2.4817, -1.4817, -0.4817,  1.5183,  2.5183,  3.5183,
           4.5183],
         [-3.4867, -2.4867, -1.4867, -0.4867,  1.5133,  2.5133,  3.5133,
           4.5133],
         [-3.4826, -2.4826, -1.4826, -0.4826,  1.5174,  2.5174,  3.5

### Test SimilarityFeeder

In [25]:
similarity_feeder = SimilarityFeeder(similarity_embedder, pos_mv_dict)

In [26]:
query = torch.tensor(2)

In [27]:
%timeit cat_ambs, IoUs = similarity_feeder(query, inputs)

1.1 ms ± 49.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### oops..

In [28]:
%timeit _ = similarity_feeder.makeIoUtensor(query, inputs)

998 µs ± 23.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
query_set= pos_mv_dict[1]['userSet']
support_set= pos_mv_dict[2]['userSet']

%timeit intersect_len = len(query_set.intersection(support_set))

325 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%timeit query_set.intersection(support_set)

317 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### The intersect is what's hogging all the time. Figured as much. Let's look into a vectorized IoU calc... 

In [31]:
set1 = {1,3,5,7,8,9,11,13,14,15}
set2 = {1,2,4,6,7,8,10,12,13,15}
array1 = torch.tensor([0,1,0,1,0,1,0,1,1,1,0,1,0,1,1,1])
array2 = torch.tensor([0,1,1,0,1,0,1,1,1,0,1,0,1,1,0,1])

In [32]:
def set_IoU(query_set, support_set):

    intersect_len = len(query_set.intersection(support_set))
    query_len = len(query_set)
    support_len = len(support_set)

    IoU = intersect_len/(query_len+support_len-intersect_len)

    return IoU

def tensor_IoU(query_tens, support_tens):
    
    intersect = torch.dot(query_tens, support_tens)
    query_len = torch.sum(query_tens)
    support_len = torch.sum(support_tens)
    
    IoU = intersect/(query_len+support_len-intersect)
    
    return IoU

In [33]:
%timeit iou = set_IoU(set1, set2)
%timeit iou = tensor_IoU(array1, array2)
%timeit torch.dot(array1, array2)
%timeit torch.sum(array1)

447 ns ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
17.3 µs ± 385 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.52 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.92 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Looks like the set is faster, maybe I vectored bad but I don't think so.. Let's scale it up and see if we get the same results: 

In [34]:
print(np.random.rand())

0.7423232213494015


In [35]:
set1 = set()
set2 = set()
tensor1 = torch.zeros((60000))
tensor2 = torch.zeros((60000))

for i in range(60000):
    prob1 = np.random.rand()
    prob2 = np.random.rand()
    thresh = 0.9
    if prob1 > thresh:
        set1.add(i)
        tensor1[i] = 1
    if prob2 > thresh:
        set2.add(i)
        tensor2[i] = 1
print(torch.sum(tensor2))
print(len(set2))

tensor(5999.)
5999


In [36]:
%timeit iou = set_IoU(set1, set2)
%timeit iou = tensor_IoU(tensor1, tensor2)

118 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
33.4 µs ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Okay, I'll go back and vectorize

In [37]:
cat_embs, IoUs = similarity_feeder(query, inputs)

### Okay, I went back and vectorized and it turned out that due to the sparsity, the set method is faster.. Continuing....

In [38]:
print(IoUs.shape)
print(cat_embs.shape)

torch.Size([8, 1])
torch.Size([8, 16])


### All the individual pieces work as expected, lets put it together

In [39]:
support_set = torch.tensor([60,1,42,23,84,45,26,74])
query = torch.tensor(2)

attention_factor = 4
similarity_factor = 8
n_movies = 100

attention_embedder = Embedder(n_factors=attention_factor, n_movies=n_movies)
similarity_embedder = Embedder(n_factors=similarity_factor, n_movies=n_movies)

attention_module = AttentionModule(attention_embedder)
similarity_feeder = SimilarityFeeder(similarity_embedder, pos_mv_dict)
similarity_module = SimilarityModule(n_factors=similarity_factor)

cat_embs, IoUs = similarity_feeder(query, support_set)
sim_out = similarity_module(cat_embs, IoUs)
att_out = attention_module(support_set)

In [40]:
print("Similarity network output:")
print(sim_out)
print(sim_out.shape)
print("Attention network output:")
print(att_out)
print(att_out.shape)

Similarity network output:
tensor([[0.6841],
        [0.6953],
        [0.6359],
        [0.6945],
        [0.6235],
        [0.6659],
        [0.6602],
        [0.6310]], grad_fn=<AddBackward0>)
torch.Size([8, 1])
Attention network output:
tensor([8.2959e-05, 1.1303e-08, 2.7887e-03, 2.3223e-01, 5.0874e-01, 1.3373e-03,
        1.0044e-01, 1.5438e-01], grad_fn=<SoftmaxBackward>)
torch.Size([8])


In [41]:
out = torch.dot(att_out, sim_out.squeeze(1))
sig = torch.nn.Sigmoid()
sig_out = sig(out)
print(out)
print(sig_out)

tensor(0.6449, grad_fn=<DotBackward>)
tensor(0.6559, grad_fn=<SigmoidBackward>)


### Load from files:

In [3]:
import pickle

# nested dict: [user]['pos'/'neg'][movieId] -> rating
train_data  = pickle.load( open( "./data/user_rating_dict_train.p", "rb" ) )
test_data   = pickle.load( open( "./data/user_rating_dict_test.p", "rb" ) )
# Dict movieId : 'title', 'genres'
movies_dict = pickle.load( open( "./data/movies_dict.p", "rb" ) )
# Dict with movieId : 'size', 'userSet'
pos_mv_dict = pickle.load( open( "./data/pos_movie2user_dict.p", "rb" ) )
neg_mv_dict = pickle.load( open( "./data/neg_movie2user_dict.p", "rb" ) )
# set of all movieId's
movieId_set = pickle.load( open( "./data/movieId_set.p", "rb" ) )

In [7]:
class AttentionFewshot(nn.Module):
    def __init__(self, pos_mv_dict, attention_factor=32, similarity_factor=8, n_movies=58958):
        """
        Parameters
        ----------
        attention_factor  : int
            size of attention embeddings
        similarity_factor : int
            size of similarity embeddings
        n_movies.         : int
            number of movies in data
          
        Forward:
            query       : torch.tensor(1)
                movieId of the query
            support_set : torch.tensor(k)
                movieId's of the support set
        
        Output
        ----------
        sig_out: Tensor(1)
            similarity value between movies
        """
        super(AttentionFewshot, self).__init__()
        
        self.attention_factor = attention_factor
        self.similarity_factor = similarity_factor
        self.n_movies = n_movies

        attention_embedder = Embedder(n_factors=attention_factor, n_movies=n_movies)
        similarity_embedder = Embedder(n_factors=similarity_factor, n_movies=n_movies)

        self.attention_module = AttentionModule(attention_embedder)
        self.similarity_feeder = SimilarityFeeder(similarity_embedder, pos_mv_dict)
        self.similarity_module = SimilarityModule(n_factors=similarity_factor)
        
        self.sig = torch.nn.Sigmoid()
        
    def forward(self, query, support_set):
        cat_embs, IoUs = self.similarity_feeder(query, support_set)
        sim_out = self.similarity_module(cat_embs, IoUs)
        att_out = self.attention_module(support_set)
        
        out = torch.dot(att_out, sim_out.squeeze(1))
        sig_out = self.sig(out)
        
        return sig_out

NameError: name 'nn' is not defined

In [158]:
support_set = torch.tensor([60,1,42,23,84,45,26,74])
query = torch.tensor([2])
print(query.shape)
print(support_set.shape)

model = AttentionFewshot(pos_mv_dict, attention_factor=4, similarity_factor=8, n_movies=100)

torch.Size([1])
torch.Size([8])


In [45]:
%timeit out = model(query, support_set)

1.29 ms ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Looks like about 75% of the time is in the IoU calc..

In [125]:
out = model(query, support_set)

In [47]:
print(out)
print(len(movies_dict))

tensor(0.6646, grad_fn=<SigmoidBackward>)
58958


In [48]:
def get_recs(model, support_set):
    predictions = []
    n_movies = 100 #len(movies_dict)
    
    for i in range(n_movies):
        out = model(torch.tensor(i), support_set)
        predictions.append(out)
        
    return predictions

In [141]:
import time

start = time.time()
recommendations = get_recs(model, support_set)
stop = time.time()
tot = stop-start
print(tot)

0.12082886695861816


In [142]:
print(recommendations)

[tensor(0.6343, grad_fn=<SigmoidBackward>), tensor(0.6345, grad_fn=<SigmoidBackward>), tensor(0.6377, grad_fn=<SigmoidBackward>), tensor(0.6362, grad_fn=<SigmoidBackward>), tensor(0.6374, grad_fn=<SigmoidBackward>), tensor(0.6347, grad_fn=<SigmoidBackward>), tensor(0.6363, grad_fn=<SigmoidBackward>), tensor(0.6391, grad_fn=<SigmoidBackward>), tensor(0.6392, grad_fn=<SigmoidBackward>), tensor(0.6344, grad_fn=<SigmoidBackward>), tensor(0.6346, grad_fn=<SigmoidBackward>), tensor(0.6408, grad_fn=<SigmoidBackward>), tensor(0.6363, grad_fn=<SigmoidBackward>), tensor(0.6361, grad_fn=<SigmoidBackward>), tensor(0.6374, grad_fn=<SigmoidBackward>), tensor(0.6343, grad_fn=<SigmoidBackward>), tensor(0.6343, grad_fn=<SigmoidBackward>), tensor(0.6364, grad_fn=<SigmoidBackward>), tensor(0.6351, grad_fn=<SigmoidBackward>), tensor(0.6370, grad_fn=<SigmoidBackward>), tensor(0.6343, grad_fn=<SigmoidBackward>), tensor(0.6352, grad_fn=<SigmoidBackward>), tensor(0.6365, grad_fn=<SigmoidBackward>), tensor(0.6

In [ ]:
from torch.utils.data import Dataset, DataLoader

class AttentionFewshotML(Dataset):
    """AttentionFewshot dataset from MovieLens20m."""

    def __init__(self, pos_movie_dict, neg_movie_dict, movieId_set, user_posneg, k=3):
        """
        Args:
            pos_movie_dict (dict): {movieId : {userId's}}
            neg_movie_dict (dict): {movieId : {userId's}}
            data (dict): {userId : {'pos' : {movieId : rating}, 'neg' : {movieId : rating} } }.
        """
        self.pos_movie_dict = pos_movie_dict
        self.neg_movie_dict = neg_movie_dict
        self.movieId_set = movieId_set
        self.user_posneg = user_posneg
        self.um_pairs = []

        for user, posneg_dict in tqdm(self.user_posneg.items()):
            pos_len = len(posneg_dict['pos'])
            if pos_len > k+1:
                for ke,va in posneg_dict['pos'].items():
                    self.um_pairs.append((user, ke))
        
        self.len = len(self.um_pairs)
        
                
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
    
        user, pos_mv = self.um_pairs[idx]
        pos_set = set(self.user_posneg[user]['pos'].keys())
        pos_set.remove(pos_mv)
        support = torch.tensor(random.sample(pos_set,3))
        pos_set.add(pos_mv)
        rand = np.random.rand()
        
        # return positive sample
        if rand > 0.5:
            pos_set.remove(pos_mv)
            
            sample = (user, torch.tensor(pos_mv), support, torch.tensor(1))
        
        # return negative sample
        else:
            neg_set = set(self.user_posneg[user]['neg'].keys())
            if neg_set:
                # stochastically sample between neg set and random movie
                prob = 1/len(neg_set)
                prob = min(1/2, prob)
                rand = np.random.rand()
                if rand > prob:
                    neg_mv = random.sample(neg_set,1)

                # This is so ugly right now... divide by 0's and whatnot
                else:
                    random_movie = self.movieId_set.symmetric_difference(pos_set)
                    neg_mv = random.sample(random_movie, 1)
            else:
                random_movie = self.movieId_set.symmetric_difference(pos_set)
                neg_mv = random.sample(random_movie, 1)
            
            sample = (user, torch.tensor(neg_mv[0]), support, torch.tensor(0))

        return sample
    
    def __random_sample__(self):
        idx = np.random.randint(self.len)
        
        user, pos_mv = self.um_pairs[idx]
        pos_set = set(self.user_posneg[user]['pos'].keys())
        pos_set.remove(pos_mv)
        support = torch.tensor(random.sample(pos_set,3))
        pos_set.add(pos_mv)
        rand = np.random.rand()
        
        # return positive sample
        if rand > 0.5:
            pos_set.remove(pos_mv)
            
            sample = (user, torch.tensor([pos_mv]), support, torch.tensor(1.))
        
        # return negative sample
        else:
            neg_set = set(self.user_posneg[user]['neg'].keys())
            if neg_set:
                # stochastically sample between neg set and random movie
                prob = 1/len(neg_set)
                prob = min(1/2, prob)
                rand = np.random.rand()
                if rand > prob:
                    neg_mv = random.sample(neg_set,1)

                # This is so ugly right now... divide by 0's and whatnot
                else:
                    random_movie = self.movieId_set.symmetric_difference(pos_set)
                    neg_mv = random.sample(random_movie, 1)
            else:
                random_movie = self.movieId_set.symmetric_difference(pos_set)
                neg_mv = random.sample(random_movie, 1)
            
            sample = (user, torch.tensor([neg_mv[0]]), support, torch.tensor(0.))

        return sample

In [185]:
train_af_dataset = AttentionFewshotML(pos_mv_dict, neg_mv_dict, movieId_set, train_data)

  0%|          | 0/130032 [00:00<?, ?it/s]

In [150]:
%timeit u, query, support_set, label = train_af_dataset.__random_sample__()

155 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [173]:
similarity_factor = 32
attention_factor = 4
n_movies = len(movieId_set)

model = AttentionFewshot(pos_mv_dict, 
                         attention_factor=attention_factor, 
                         similarity_factor=similarity_factor, 
                         n_movies=n_movies)

u, query, support_set, label = train_af_dataset.__random_sample__()
print(query)
print(query.shape)
print(support_set)
print(support_set.shape)
out = model(query, support_set)
print(out)

tensor([13240])
torch.Size([1])
tensor([ 585, 2211,  314])
torch.Size([3])
support embeds: torch.Size([3, 32])
query embeds: torch.Size([3, 32])
cat embeds: torch.Size([3, 64])
tensor(0.6633, grad_fn=<SigmoidBackward>)


In [5]:
# Grabbed from: https://discuss.pytorch.org/t/check-gradient-flow-in-network/15063/8

def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])

In [6]:
import torch.optim as optim
from tqdm.notebook import trange

alpha = -0.05
c0 = 2000

train_iters = 1000000  # number of training steps
batch_size = 50      # number of steps per batch
val_iters = 10000      # number of steps before evaluating

similarity_factor = 32
attention_factor = 4
n_movies = len(movieId_set)

model = AttentionFewshot(pos_mv_dict, 
                         attention_factor=attention_factor, 
                         similarity_factor=similarity_factor, 
                         n_movies=n_movies)

train_af_dataset = AttentionFewshotML(pos_mv_dict, neg_mv_dict, movieId_set, train_data)
test_af_dataset = AttentionFewshotML(pos_mv_dict, neg_mv_dict, movieId_set, test_data)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

batch_train_loss = 0
batch_val_loss = 0
batch_train_losses = []
batch_val_losses = []

t = trange(train_iters, desc=f'Train Loss: {0}   Val Loss: {0}', leave=True)
for i in t:
    
    model.train()
    
    u, query, support_set, label = train_af_dataset.__random_sample__()
    
    # Loss weighting mechanism
    Iu_sup = len(support_set)
    Iu_all = len(train_data[u]['pos'])
    cu = c0*(Iu_sup**alpha)/(Iu_all**alpha)
    
    pred = model(query, support_set)
    loss = criterion(pred, label)
    
    batch_train_loss += cu*loss.item()
    
    loss.backward()

    # Track loss landscape over network:
    plot_grad_flow(model.parameters())
    
    if (i+1) % batch_size == 0:
        optimizer.step()
        optimizer.zero_grad()
        _batch_train_loss = batch_train_loss/batch_size
        
        t.set_description(f'Train Loss: {batch_train_loss}   Val Loss: {batch_val_loss}')
        t.update()
        
    if (i+1) % val_iters == 0:
        batch_val_loss = 0
        for j in range(batch_size):
             
            model.eval()

            with torch.no_grad():
                u, query, support_set, label = test_af_dataset.__random_sample__()
                pred = model(query, support_set)
                loss = criterion(pred, label)
                batch_val_loss += loss.item()/batch_size
        
        batch_val_losses.append(batch_val_loss)
        batch_train_losses.append(_batch_train_loss)
    
        t.set_description(f'Train Loss: {batch_train_loss}   Val Loss: {batch_val_loss}')
        t.update()
        
        torch.save(model.state_dict(), f"./models/xp2_{i}.pt")

    batch_train_loss = 0
           
        
    
    # between the way I built the model and the per-user custom loss, I decided to do manual batch aggregation
    # Support for Dataloader and batching may be added in future work
    
    
    

NameError: name 'AttentionFewshot' is not defined

In [4]:
import csv

filename = './losses.csv'

with open(filename) as f:
    reader = csv.reader(f)
    data = list(reader)

train_loss = [x[0] for x in data]
print(train_loss)

['35.221969239765365', '40.849517897919554', '34.329036256865194', '33.11454249873748', '34.910168477301305', '35.72636311544923', '36.81274070026958', '34.91981843289303', '15.957053814546418', '59.05068009194659', '34.849570375505685', '36.33342285570924', '35.90862323752629', '33.573414835292624', '34.20720435539711', '33.6714145595823', '38.06797290095458', '33.51636756262111', '72.16021757975376', '33.36744039839019', '36.79223547637299', '35.812533715423484', '30.963440296912182', '21.824576636739042', '17.88719043952244', '44.98665887717735', '35.641981863477525', '31.47383611428922', '35.07568477656124', '19.159610720912283', '35.21698685562268', '35.94407226650521', '59.23253851800186', '37.46640689576068', '35.688861182679126', '21.89741285509776', '37.095027779018395', '15.660940914745398', '16.185336018390537', '14.79685966899514', '15.28925139883554', '30.83102513690251', '32.483148730751005', '34.21214017994138', '37.8897152840095', '36.33666328019765', '21.45951430299210

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss)

In [183]:
target = torch.empty(3).random_(2)
print(target)

t = trange(100, desc='Bar desc', leave=True)
for i in t:
    t.set_description(f"Bar desc {i}")
#     t.refresh() # to show immediately the update
    #sleep(0.01)

tensor([0., 1., 0.])


Bar desc:   0%|          | 0/100 [00:00<?, ?it/s]

In [154]:
print(len(train_data[1]['pos']))
print(len(movieId_set))

60
58958


In [88]:
# Testing for the custom Dataset functions
import random
_train_data = []
for i in range(15):
    user = data[i][0]
    pos_mv = data[i][1]
    
    pos_set = set(train_data[user]['pos'].keys())
    pos_set.remove(pos_mv)
    support = torch.tensor(random.sample(pos_set,3))

    neg_set = set(train_data[user]['neg'].keys())
    neg_mv = random.sample(neg_set,1)
    
    _train_data.append((user, torch.tensor(pos_mv), support, torch.tensor(1)))
    _train_data.append((user, torch.tensor(neg_mv[0]), support, torch.tensor(0)))
    

In [89]:
[print(i) for i in _train_data]

(110941, tensor(15), tensor([  351,  4772, 18711]), tensor(1))
(110941, tensor(21373), tensor([  351,  4772, 18711]), tensor(0))
(110941, tensor(31), tensor([ 9333, 20450, 20602]), tensor(1))
(110941, tensor(21373), tensor([ 9333, 20450, 20602]), tensor(0))
(110941, tensor(46), tensor([1566, 9343, 1620]), tensor(1))
(110941, tensor(21476), tensor([1566, 9343, 1620]), tensor(0))
(110941, tensor(49), tensor([18711, 15507, 43259]), tensor(1))
(110941, tensor(21476), tensor([18711, 15507, 43259]), tensor(0))
(110941, tensor(108), tensor([ 7236,  2766, 13405]), tensor(1))
(110941, tensor(24526), tensor([ 7236,  2766, 13405]), tensor(0))
(110941, tensor(250), tensor([7743,  734, 1191]), tensor(1))
(110941, tensor(21373), tensor([7743,  734, 1191]), tensor(0))
(110941, tensor(257), tensor([7743, 4122, 1304]), tensor(1))
(110941, tensor(1466), tensor([7743, 4122, 1304]), tensor(0))
(110941, tensor(289), tensor([34319,  1969,   475]), tensor(1))
(110941, tensor(1466), tensor([34319,  1969,   47

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

{4, 5, 6, 7}


In [61]:
movies = set(train_data[data[0][0]]['pos'].keys())
print(movies)
print()
movies.remove(15)
print(movies)
print()
print(set(train_data[data[0][0]]['pos'].keys()))

{10244, 3083, 15, 34319, 3602, 534, 1559, 536, 4119, 4122, 6169, 7196, 2077, 1566, 31, 9752, 11807, 12318, 18975, 2084, 1062, 4135, 1065, 13353, 46, 1070, 1583, 49, 9775, 21555, 5174, 1591, 7222, 4158, 4670, 2624, 7742, 7743, 17474, 7236, 581, 7237, 9284, 12870, 585, 4682, 14921, 44180, 589, 1101, 1614, 40018, 1620, 38998, 3159, 600, 13405, 20574, 12895, 15967, 15459, 1640, 19562, 108, 21100, 2670, 17522, 9333, 20602, 637, 9343, 2177, 1666, 3207, 12423, 1164, 17036, 1166, 1168, 2705, 1170, 11923, 15507, 2197, 17046, 1175, 4759, 11928, 17432, 1179, 18587, 17565, 1182, 1183, 4768, 14495, 15518, 28831, 4772, 31477, 1190, 1191, 11432, 1713, 11444, 4278, 1207, 2233, 1213, 2238, 5310, 10435, 1225, 10954, 9420, 1230, 2766, 5840, 21713, 20180, 1237, 734, 20702, 17641, 1258, 6379, 5877, 13558, 19703, 22773, 4857, 762, 250, 764, 5883, 13565, 41718, 43259, 257, 1795, 3845, 11014, 2312, 2824, 2826, 14090, 28430, 9999, 6416, 21782, 4887, 1304, 11544, 18711, 18715, 28439, 289, 17187, 292, 2855, 4279

# TODO: Implement NDGC, MAP@K, MAR@K, Personalization

# THEN TRAIN!!!

In [42]:
id2title_dict_2 = dict(zip(movies_df2['movieId'], movies_df2['title']))
id2title_dict_3 = dict(zip(movies_df3['movieId'], movies_df3['title']))
id2title_dict_4 = dict(zip(movies_df4['movieId'], movies_df4['title']))

In [43]:
new_df['col1'] = rating_df['movieId'].apply(lambda movieId: id2title_dict_2[movieId])

NameError: name 'new_df' is not defined